In [105]:
import pickle
import pandas as pd
import numpy as np
import re
import itertools
import json

from konlpy.tag import Komoran

In [72]:
data_path = "/home/ai-sjjy/workspace/juyoung/JasoseoAI_project/3.data/Keyword/"
src_path = "/home/ai-sjjy/workspace/juyoung/JasoseoAI_project/2.src/Keyword_AI/"

In [8]:
with open(data_path+'JobKorea_split_content.pickle', 'rb') as p:
    jobkorea = pickle.load(p)
    
with open(data_path+'finalCategory_dic.pickle', 'rb') as f:
    keywords = pickle.load(f)

with open(data_path+'CompanyKeywordData.json', 'r', encoding='utf-8') as f: # 회사-[질문:답변] => dict 형식
    dataset = json.load(f)

In [9]:
len(dataset)

172

In [10]:
len(keywords)

48

In [23]:
companys = list(dataset.keys())

### content split 가져오기

In [16]:
jobkorea = jobkorea[['company_name', 'split_content']]

In [25]:
jobkorea

,company_name,split_content
0,경동나비엔,"[[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다., 조금 늦은 나이..."
1,포스코인터내셔널,[[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도...
2,하나은행,"[[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력저는 어려서 배운 컴퓨터 강..."
3,조선비즈,[[저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 어려움을 덜어낸 경...
4,cjenm,"[[음악을 사랑합니다., 음악뿐만 아니라 문화산업 전반에 깊은 관심이 있습니다., ..."
...,...,...
6626,삼화페인트공업,"[[남들을 배려하며 착하게 사는 것이 옳다고 배우며 살아왔기 때문에, 이타적인 성향..."
6627,lg디스플레이,"[[2013년 여름, 천호식품에서 아르바이트를 했었습니다., 제품 생산에서부터 포장..."
6628,엘에스전선,"[[선배들의 추천을 받아 13년도 동아리 회장을 맡았습니다., 당시 숫기가 없어 동..."
6629,soil,"[[최초를 시도한 기업은 많습니다., 하지만 최초보다 더욱 중요한 것이 최고라고 생..."


In [19]:
jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub(" +","",x.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", "")))
jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub("[^가-힣a-z0-9]", "", x))

<ipython-input-19-3d12f3f8d7bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub(" +","",x.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", "")))
<ipython-input-19-3d12f3f8d7bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub("[^가-힣a-z0-9]", "", x))


In [20]:
jobkorea.head(3)

,company_name,split_content
0,경동나비엔,"[[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다., 조금 늦은 나이..."
1,포스코인터내셔널,[[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도...
2,하나은행,"[[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력저는 어려서 배운 컴퓨터 강..."


In [61]:
group = jobkorea.groupby('company_name')
c = 0
jobkorea_group = dict()
for idx, g in group:
    sentences = []
    for sentence in g.split_content.tolist():
        for s in sentence:
            sentences.append(s)
    jobkorea_group[idx] = sentences

In [68]:
split_company_keywords = dict()
for key, value in jobkorea_group.items():
    if key in companys:
        split_company_keywords[key] = {'answer' : value, 'keywords' : dataset[key]['keywords']}

In [69]:
len(split_company_keywords)

172

In [71]:
with open(data_path+'CompanyKeywordData2.json', 'w', encoding='utf-8') as f: # 회사-[질문:답변] => dict 형식
    json.dump(split_company_keywords, f, indent="\t")

## 데이터 전처리

In [75]:
with open(data_path+'CompanyKeywordData2.json', 'r') as f: # 회사-[질문:답변] => dict 형식
    dataset = json.load(f)

In [125]:
# 데이터 정제

komo = Komoran()
def clean_content(sentences, stopwords):
    sentences_words = []
    
    for sentence in sentences:
        # 정규 표현식 필터 1 - 기호 제거(한글, 숫자, 영어 => )
        sentence = re.sub('[^a-f가-힣 ]', "", sentence.lower())
        
        new_words = []
        for word, pos in komo.pos(sentence):
            # "NNG","NNP", "VV", "VA", "VCP", "VCN", "XPN", "XR", "SL", "NF", "NV", "NA"
            if pos in ("NNG", "NNP", "SL"):
                if word not in stopwords:
                    if pos != "SL": # stop word 
                        sentences_words.append(word)
                
        #sentences_words.append(new_words)
        
    return sentences_words

#### stop word 찾아서 추가하기 및 코모란 단어 추가

In [97]:
komoran_words_add = set()

In [ ]:
space_error = set("따뜻한 말 한마디")

In [90]:
# stop word 추출
f = open(src_path+'stopwords.txt', 'r')
stopwords = set()
while True:
    line = f.readline()
    if not line:break
    stopwords |= set(line[:-1])
f.close()

In [ ]:
# komoran add 단어 추출
f = open(src_path+'add_words.txt', 'r')
stopwords = set()
while True:
    line = f.readline()
    if not line:break
    stopwords |= set(line[:-1])
f.close()

In [99]:
for company, value in dataset.items():
    print(company)
    for content in value['answer']:
        print(clean_content(content, stopwords))
        result = input("불용어 추가 : ")
        if result :
            if "," not in result: stopwords |= set(result)
            else: 
                for r in result.split(","):
                    stopwords |= set(r.strip())
        result = input("코모란 단어 추가 : ")
        if result : 
            if "," not in result: komoran_words_add |= set(result)
            else:
                 for r in result.split(","):
                    komoran_words_add |= set(r.strip())

db금융투자
[['성장', '기조', '장기', '저금리', '기조', '지속', '과거', '현재', '미국', '금리', '상승', '시장', '관하', '기준', '바꾸', '모습'], ['미국', '유럽', '발', '경기', '회복', '흐름', '나타나', '시작', '글로벌', '시장', '상승', '반전', '기대', '모습'], ['국내', '증권사', '수익', '차지', '브로커', '리지', '영역', '성장', '여력', '고민', '안기'], ['증권사', '부문', 'fcc', '부문', '신설', '구조', '재편', '새롭', '활력', '불어넣'], ['기조', '힘입', '동부증권', '새롭', '앞길', 'fcc', '부문', '힘', '지원']]


불용어 추가 :  
코모란 단어 추가 :  


[['장점', '분석력', '팀워크'], ['대학', '생활', '수학', '금융공학', '논리력', '문제', '해결', '능력'], ['대학', '가치투자', '학회', '기초', '산업', '지식', '쌓', '인원', '활동', '다양', '사람', '모이', '만들', '시너지', '대하', '깨닫'], ['바탕', '자산운용', '투자자', '문사', '인턴', '생활', '성공', '수행'], ['규모', '자산운용', '근무', '바이오', '반도체', '장비', '섹터', '맡', '산업', '브리핑', '기업', '분석', '수행'], ['바이오', '섹터', '특성', '전공자', '이상', '기업', '가지', '잠재', '기술', '대하', '정확', '파악', '어렵', '해외', '저널', '논문', '검색', '교내', '교수', '기술', '관하', '물어보', '타당성', '검증', '방면', '노력'], ['바탕', '정확', '리포트', '작성', '투자', '위험성', '대하', '경고'], ['남', '부탁', '거절', '단점'], ['부탁', '쌓이', '방해', '보완', '생각'], ['부탁', '이유', '상대방', '믿', '사람', '생각', '유지', '노력']]


불용어 추가 :  
코모란 단어 추가 :  


[['조각가', '남기'], ['경험', '돈'], ['동의', '경험', '중시', '살아오'], ['피아노', '유도', '테니스', '수영', '성악', '배우', '자라', '다양', '대외', '활동', '참여', '도전', '경험', '군대', '훈련', '참가'], ['기본', '미군', '부사관', '진급', '일종', '진급', '시험'], ['카투사', '동료', '모병제', '지원', '미군', '원만', '지원', '훈련', '프로그램'], ['주기', '바깥세상', '나가', '매일', '매일', '교육', '훈련', '프로그램', '이루어지', '빡빡'], ['인정받', '카투사', '훈련', '프로그램', '신청', '이유', '미군', '수료', '으스대'], ['주간', '훈련', '마치', '돌아오', '분대장', '임명', '부대원', '대표', '한국군', '미군', '사이에서', '가교', '역할'], ['경험주의', '사고방식', '힘들', '시기', '경험', '배우', '생각', '스트레스', '완화', '나아가', '발전', '도모', '유연', '태도', '지니', '도와주'], ['다방면', '경험', '다양', '사람', '공감', '가능', '배우']]


...

불용어 추가 :  게, 갖, 해보, 임하, 여쭈, 토대
코모란 단어 추가 :  뇌병변, 허순남


[['넘', '학생', '지도', '학원', '팀장', '역임'], ['학생', '출결', '체크', '진도', '관리', '강사', '관리', '학부모', '상담', '영', '어부', '학부', '사이', '시간표', '협상', '조율', '마무리', '원장', '높', '신뢰'], ['지각', '빈틈없', '모습', '근무', '강사', '학생', '철저'], ['학원', '리더십', '가치', '팀원', '지시', '모범', '따라오', '만들', '사실', '깨닫'], ['통계학', '전공', '신장', '데이터', '분석', '능력', '동부', '화재', '지점', '위치', '시장', '분석', '적용'], ['리더십', '데이터', '분석', '능력', '접목', '고객', '니즈', '충족', '피드백', '지속', '제공', '자신'], ['느리', '미학', '메모', '스케줄', '애용', '성격'], ['동아리', '활동', '봉사', '활동', '대외', '활동', '인턴', '업무', '학과', '프로젝트', '공익', '근무', '다채', '활동', '진행', '동시', '처리', '순간'], ['생기', '최대한', '수행', '팀원', '부담', '느끼'], ['상대방', '입장', '고려', '일정', '타협', '진행']]


불용어 추가 :  넘
코모란 단어 추가 :  


[['통계학', '전공', 'ace', '관심', '향후', '보험', '산업', '발전', '가능', '인지'], ['멘토링', '진로', '설계', '보험사', '비전', '구체', '평소', '스트레스', '상황', '무던', '남', '끈기', '영업', '관리', '직무', '수행', '자신', '동부', '화재', '지원'], ['스펀지', '사원', '신입', '입사', '후', '초기', '동부', '화재', '기업', '문화', '녹', '들어가', '노력', '업무', '파악', '매진'], ['스펀지', '물', '흡수', '선배', '업무', '노하우', '지식', '흡수', '자세', '갖추'], ['겸손', '마음', '조금', '받아들이', '성실', '배우', '노력', '모습', '드리'], ['아들', '영업', '관리', '보험', '영업', '분야', '분야', '비하', '이타', '마인드', '중요', '생각'], ['아들', '영업', '관리인', '노력'], ['보험', '절대', '어렵', '아깝', '재미있', '일상', '생활', '용품', '느낌', '고객', '전하'], ['보험', '상품', '구조', '이해', '힘들', '고객', '시간', '마련', '명쾌', '전문', '지식', '바탕', '강의', '고객', '어머니', '생각', '지원']]


KeyboardInterrupt: Interrupted by user

In [104]:
stopwords

{'ff',
 '가',
 '가을',
 '가져오',
 '가지',
 '각',
 '간',
 '갖',
 '같',
 '개',
 '걸',
 '것',
 '게',
 '겨울',
 '경',
 '고',
 '관하',
 '그',
 '그렇',
 '그중',
 '금',
 '기',
 '기반',
 '까',
 '끝내',
 '나',
 '날',
 '내',
 '넘',
 '녀',
 '년',
 '놓',
 '니',
 '다',
 '단위',
 '당시',
 '당일',
 '대',
 '대부분',
 '대신',
 '대하',
 '더',
 '데',
 '도',
 '동안',
 '되',
 '두',
 '뒤',
 '드',
 '들',
 '들이',
 '등',
 '따',
 '때',
 '떤',
 '떻',
 '또',
 '뛰',
 '뜨',
 '람',
 '러',
 '런',
 '렇',
 '로',
 '르',
 '른',
 '리',
 '만',
 '많',
 '말',
 '맞',
 '맡',
 '매',
 '면',
 '명',
 '모',
 '못',
 '문',
 '반면',
 '받',
 '번',
 '보',
 '보내',
 '보이',
 '봄',
 '사',
 '살',
 '상',
 '생',
 '소',
 '속',
 '손',
 '수',
 '시',
 '신',
 '실',
 '싶',
 '씨',
 '아',
 '아니',
 '안',
 '않',
 '알',
 '앞',
 '앞으로',
 '어',
 '어떻',
 '얻',
 '업',
 '없',
 '여',
 '여름',
 '여쭈',
 '오',
 '오래전',
 '올해',
 '우',
 '원',
 '월',
 '위',
 '위하',
 '의',
 '이',
 '이러',
 '이렇',
 '이후',
 '일',
 '임하',
 '있',
 '자',
 '잘',
 '저',
 '적',
 '전',
 '전체',
 '점',
 '정',
 '제',
 '제가',
 '좀',
 '좁',
 '좋',
 '주',
 '중',
 '지',
 '집',
 '찾',
 '최',
 '최근',
 '최종',
 '층',
 '크',
 '키',
 '토대',
 '통',
 '통하',
 '하',
 '한',
 '해내',
 '

In [100]:
f = open(src_path+'stopwords.txt', 'w')

for word in stopwords:
    f.write(word+"\n")
f.close()

In [102]:
komoran_words_add = {"계리사", "뇌병변", "동부화재", "리뉴얼", "융복합", "허순남", "스타트업"}

In [103]:
f = open(src_path+'add_words.txt', 'w')

for word in stopwords:
    f.write(word+"\n")
f.close()

#### 단어 처리

In [126]:
new_dataset = dict()
for company, value in dataset.items():
    temp = []
    for content in value['answer']:
        temp.append(clean_content(content, stopwords))
    new_dataset[company] = {'preprocess' : temp, 'keywords' : value['keywords']}

In [128]:
for company, value in new_dataset.items():
    print(value['preprocess'][0])
    break

['성장', '기조', '장기', '저금리', '기조', '지속', '과거', '현재', '미국', '금리', '상승', '시장', '기준', '모습', '미국', '유럽', '발', '경기', '회복', '흐름', '시작', '글로벌', '시장', '상승', '반전', '기대', '모습', '국내', '증권사', '수익', '차지', '브로커', '리지', '영역', '성장', '여력', '고민', '증권사', '부문', '부문', '신설', '구조', '재편', '활력', '기조', '동부증권', '앞길', '부문', '힘', '지원']


In [193]:
with open(data_path+'CompanyKeywordData_Preprocess.json', 'w', encoding='utf-8') as f: # 회사-[질문:답변] => dict 형식
    json.dump(new_dataset, f, indent="\t")

### Word2vec fine tuning

In [129]:
all_data = []
for company, value in new_dataset.items():
    for v in value['preprocess']:
        all_data.append(v)

In [132]:
all_data[1]

['장점',
 '분석력',
 '팀워크',
 '대학',
 '생활',
 '수학',
 '금융공학',
 '논리력',
 '문제',
 '해결',
 '능력',
 '대학',
 '가치투자',
 '학회',
 '기초',
 '산업',
 '지식',
 '인원',
 '활동',
 '사람',
 '시너지',
 '바탕',
 '자산운용',
 '투자자',
 '문사',
 '인턴',
 '생활',
 '성공',
 '수행',
 '규모',
 '자산운용',
 '근무',
 '바이오',
 '반도체',
 '장비',
 '섹터',
 '산업',
 '브리핑',
 '기업',
 '분석',
 '수행',
 '바이오',
 '섹터',
 '특성',
 '전공자',
 '이상',
 '기업',
 '잠재',
 '기술',
 '파악',
 '해외',
 '저널',
 '논문',
 '검색',
 '교내',
 '교수',
 '기술',
 '타당성',
 '검증',
 '방면',
 '노력',
 '바탕',
 '리포트',
 '작성',
 '투자',
 '위험성',
 '경고',
 '남',
 '부탁',
 '거절',
 '단점',
 '부탁',
 '방해',
 '보완',
 '생각',
 '부탁',
 '이유',
 '상대방',
 '사람',
 '생각',
 '유지',
 '노력']

In [187]:
import gensim
# Word2vec model load and save
model = gensim.models.Word2Vec(src_path+'word2vec_pre-model/ko.bin')
model.wv.save_word2vec_format(src_path+'word2vec_pre-model/ko.bin.gz', binary=True)

In [179]:
model2 = gensim.models.Word2Vec(size=100, min_count=2)
model2.build_vocab(all_data)

In [180]:
total_examples = model2.corpus_count

In [181]:
model2.build_vocab([list(model.wv.vocab.keys())], update=True)
model2.intersect_word2vec_format(src_path+'word2vec_pre-model/ko.bin.gz', binary=True)

In [182]:
len(model2.wv.vocab)

11460

In [189]:
model2.train(
    all_data,
    total_examples = total_examples,
    epochs = 100 # model2.iter
)

(54343097, 57943600)

In [190]:
model2.most_similar('증권사')

<ipython-input-190-73a2903f577a>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model2.most_similar('증권사')


[('패러다임', 0.43787723779678345),
 ('뱅킹', 0.4323086738586426),
 ('예치', 0.3967936336994171),
 ('전망', 0.3937190771102905),
 ('금융 서비스', 0.39142340421676636),
 ('단행', 0.37949323654174805),
 ('수수료', 0.36361464858055115),
 ('경남은행', 0.3616281747817993),
 ('수요와 공급', 0.36073893308639526),
 ('뱅크', 0.3586363196372986)]

In [194]:
len(model2.wv.vocab)

11460

In [192]:
model2.save(src_path+"word2vec_pre-model/ko_new.bin")